In [5]:
from __future__ import print_function
from hyperopt import Trials, STATUS_OK, tpe, rand
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential
from keras.utils import np_utils, to_categorical 
from sklearn.metrics import accuracy_score
from hyperas import optim
from hyperas.distributions import choice, uniform
from keras import optimizers

def CollectData():
    import pymongo
    #client = pymongo.MongoClient("mongodb+srv://testuser:<testuser>@cluster0-ptheg.azure.mongodb.net/test?retryWrites=true&w=majority")
    myclient = pymongo.MongoClient("mongodb://localhost:27017/")
    mydb = myclient['SampleDB']
    myCollection = mydb['StockMarketCollection']
    
    import pandas as pd
    import numpy as np
    cursor = myCollection.find()
    StockDF =  pd.DataFrame(list(cursor))
    StockDF_sample = StockDF[['date', 'high', 'low', 'open', 'volume', 'close']]
    StockDF_sample['date'] = StockDF_sample['date'].apply(lambda x: pd.to_datetime(x))
    
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    minDate = min(StockDF_sample['date'])
    StockDF_sample['date_delta'] = (StockDF_sample['date'] - minDate).dt.days.astype(int)
    StockDF_sample.drop('date', axis=1, inplace=True)
    StockDF_sample['C2O'] =  StockDF_sample['close'] - StockDF_sample['open']
    StockDF_sample['C2O'] = np.round(StockDF_sample['C2O'], 2)
    
    threshold = 0.5
    def MarkThreeLayerProfit(x):
        if x < -threshold:
            return 2
        elif x > threshold:
            return 1
        elif x >= -threshold and x <= threshold:
            return 0
        else:
            return "-99"

    StockDF_sample['profit'] = StockDF_sample['C2O'].apply(lambda x: MarkThreeLayerProfit(x))

    Scaled_Stock = scaler.fit_transform(StockDF_sample[['high', 'low', 'open', 'volume', 'close']])
    scaled_Stock_df = pd.DataFrame(Scaled_Stock, 
                               columns=['high', 'low', 'open', 'volume', 'close'])
    scaled_Stock_df['profit']= StockDF_sample['profit']

    X = scaled_Stock_df.drop('profit', axis=1)
    Y = to_categorical(scaled_Stock_df['profit'], dtype='int')
    from sklearn.model_selection import train_test_split
    X_train, X_val, Y_train, Y_val = train_test_split(X,Y, test_size = 0.3, random_state =4567)
    return X_train, X_val, Y_train, Y_val

def create_model(X_train, Y_train, X_val, Y_val):
    from keras import models
    from keras import layers
    import numpy as np

    model = models.Sequential()
    model.add(layers.Dense({{choice([np.power(2, 5), np.power(2, 6), np.power(2, 7)])}}, input_shape=(5,)))
    model.add(LeakyReLU(alpha={{uniform(0.5, 1)}}))
    model.add(Dropout({{uniform(0.5, 1)}}))
    model.add(layers.Dense({{choice([np.power(2, 3), np.power(2, 4), np.power(2, 5)])}}))
    model.add(LeakyReLU(alpha={{uniform(0.5, 1)}}))
    model.add(Dropout({{uniform(0.5, 1)}}))
    model.add(layers.Dense(3, activation='softmax'))

    from keras import callbacks
        
    reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                  patience=5, min_lr=0.001)


    model.compile(optimizer={{choice(['rmsprop', 'adam', 'sgd'])}},
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    model.fit(X_train,
              Y_train,
              epochs={{choice([25, 50, 75, 100])}},
              batch_size={{choice([16, 32, 64])}},
              validation_data=(X_val, Y_val),
              callbacks=[reduce_lr])

    score, acc = model.evaluate(X_val, Y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=CollectData,
                                          algo=tpe.suggest,
                                          max_evals=15,
                                          trials=Trials(),
                                          notebook_name='MongoDB_Find_Feed_to_keras_HyperParam')
    
   
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_val, Y_val))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    best_model.save('StockPrediction.h5')

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    from hyperopt import Trials, STATUS_OK, tpe, rand
except:
    pass

try:
    from keras.layers.core import Dense, Dropout, Activation
except:
    pass

try:
    from keras.layers.advanced_activations import LeakyReLU
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.utils import np_utils, to_categorical
except:
    pass

try:
    from sklearn.metrics import accuracy_score
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    from keras import optimizers
except:
    pass

try:
    import pymongo
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    from sklearn.preprocessing import MinMaxScaler
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:


C:\Personnel\D\DataSciencePortfolio\finance\temp_model.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  StockDF_sample['date'] = StockDF_sample['date'].apply(lambda x: pd.to_datetime(x))
C:\Personnel\D\DataSciencePortfolio\finance\temp_model.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  StockDF_sample['date_delta'] = (StockDF_sample['date'] - minDate).dt.days.astype(int)
C:\Users\ssreekan\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy

  0%|                                                                             | 0/15 [00:00<?, ?it/s, best loss: ?]

WARNING - From C:\Users\ssreekan\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 70 samples, validate on 30 samples                                                                            
Epoch 1/50                                                                                                             
16/70 [=====>........................]                                                                                 
 - ETA: 5s - loss: 1.2225 - acc: 0.3750                                                                                
                                                                                                                       
32/70 [============>.................]                                                                                 
 - ETA: 2s - loss: 1.4642 - acc: 0.3438                                                                                
                                                                                                                       
70/70 [==============================]  

...

 - ETA: 0s - loss: 1.3194 - acc: 0.3125                                                                                
                                                                                                                       
70/70 [==============================]                                                                                 
 - 0s 543us/step - loss: 1.2905 - acc: 0.3143 - val_loss: 1.1189 - val_acc: 0.2000                                     

Epoch 14/100                                                                                                           
16/70 [=====>........................]                                                                                 
 - ETA: 0s - loss: 1.2261 - acc: 0.2500                                                                                
                                                                                                                       
70/70 [==============================] 

Epoch 36/100                                                                                                           
16/70 [=====>........................]                                                                                 
 - ETA: 0s - loss: 1.1164 - acc: 0.4375                                                                                
                                                                                                                       
70/70 [==============================]                                                                                 
 - 0s 557us/step - loss: 1.0932 - acc: 0.5000 - val_loss: 1.1408 - val_acc: 0.2333                                     

Epoch 37/100                                                                                                           
16/70 [=====>........................]                                                                                 
 - ETA: 0s - loss: 1.2089 - acc: 0.4375

70/70 [==============================]                                                                                 
 - 0s 443us/step - loss: 1.0971 - acc: 0.4000 - val_loss: 1.1412 - val_acc: 0.2667                                     

Epoch 59/100                                                                                                           
16/70 [=====>........................]                                                                                 
 - ETA: 0s - loss: 1.3090 - acc: 0.1875                                                                                
                                                                                                                       
70/70 [==============================]                                                                                 
 - 0s 472us/step - loss: 1.1510 - acc: 0.3000 - val_loss: 1.1396 - val_acc: 0.2667                                     

Epoch 60/100                          

 - ETA: 0s - loss: 0.9894 - acc: 0.5625                                                                                
                                                                                                                       
70/70 [==============================]                                                                                 
 - 0s 572us/step - loss: 1.1006 - acc: 0.3857 - val_loss: 1.1465 - val_acc: 0.3000                                     

Epoch 82/100                                                                                                           
16/70 [=====>........................]                                                                                 
 - ETA: 0s - loss: 1.1256 - acc: 0.3750                                                                                
                                                                                                                       
70/70 [==============================] 

NameError: name 'X_val' is not defined